In [1]:
%matplotlib notebook
import os
from adm_library import *

In [5]:
fname = 'files/Cal 5DoF inputs 20230509.xlsx'
sheetname = 'Input for Tyler analysis'

adm_log_filepath = 'files/ADM Ops Log.xlsx'

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=1,usecols='B:N')

focal_length = spreadsheet.iloc[0][0]-110 # focal length is track length -110mm, per TG
sMPA_to_WCS = spreadsheet.iloc[3,:3].astype(float)
sMPA_angle_to_WCS_deg = spreadsheet.iloc[6,:3].astype(float)
GSA_angle_WCS_deg = spreadsheet.iloc[7][0]
sMask=np.array(spreadsheet.iloc[10,1:4]).astype(float)

spreadsheet = pd.read_excel(fname,
                            sheet_name=sheetname,skiprows=15,usecols='B:N')
pose_actual = spreadsheet.iloc[0:5,0:6]
pose_actual = pose_actual.set_index('Name').sort_index()
pose_actual.columns = ['X','Y','Z','Rx','Ry']
X_5DOF,Y_5DOF,Z_5DOF,rx_deg_5DOF,ry_deg_5DOF = [[*pose_actual['X'].values.astype(float)],[*pose_actual['Y'].values.astype(float)],[*pose_actual['Z'].values.astype(float)],
                                                [*pose_actual['Rx'].values.astype(float)],[*pose_actual['Ry'].values.astype(float)]]


In [6]:
#Generate a dataframe where we will store some parameters for each pose
columns = ['X','Y','Z','Rx','Ry']
df = pose_actual
df['color']='crimson'

gsa_rot = R.from_euler('x',GSA_angle_WCS_deg, degrees=True)
df.loc['sMask',['X','Y','Z']] = gsa_rot.apply(sMask)
df.loc['sMask','color'] = 'g'

df.loc['sMPA',['X','Y','Z']] = sMPA_to_WCS.values.astype(float)
df.loc['sMPA','color']='purple'

for pose in df.index:
    update_uvec(df,pose,length=focal_length,rotangle=GSA_angle_WCS_deg)
    if 'PDI' in pose:
        df.loc[pose,'color'] = 'yellow'

# create normal vector to sMPA
vec = np.array([0,0,1])
rotmat = R.from_euler('XYZ',sMPA_angle_to_WCS_deg, degrees=True)
df.loc['sMPA',['uvec_X','uvec_Y','uvec_Z']] = rotmat.apply(vec)

df.loc['sMPA',['Rx', 'Ry', 'Rz']] = sMPA_angle_to_WCS_deg.values.astype(float)

ax = plot_poses(df)
plot_sMPA(df, ax)
df

<IPython.core.display.Javascript object>

X           Y           Z         Rx         Ry    color  \
Name                                                                       
PR1      0.991107    0.115821    0.754433   0.021212   0.045726  crimson   
PR2   -109.485707  -46.959858    23.47922  -4.915799  10.693732  crimson   
PR3    111.329338  -46.602151   23.152164  -4.873486 -10.586882  crimson   
PR4    103.162412   58.543344   31.771468   5.291407 -10.304568  crimson   
PR5   -101.345145   58.305984   31.898431   5.265896  10.417492  crimson   
sMask       1.469  115.184948  588.226533        NaN        NaN        g   
sMPA        2.022     248.477    1268.761 -24.302000   0.043600   purple   

           uvec_X      uvec_Y       uvec_Z      Rz  
Name                                                
PR1      1.031195  248.370010  1268.027256     NaN  
PR2    239.765216  350.383100  1220.379520     NaN  
PR3   -237.396982  349.604227  1221.065684     NaN  
PR4   -231.135803  128.737997  1264.746858     NaN  
PR5    233.640931  129.254531  1264.233772     NaN  
sMask         NaN         NaN          NaN     NaN  
sMPA     0.000761    0.411546     0.911389  0.1048

# Now point to an arbitrary location in the sMPA plane

In [11]:
coords = dict({'center':np.array([100.,0.,0.])})

location = 'center'

rotmat = R.from_euler('XYZ',df.loc['sMPA',['Rx','Ry','Rz']].values.astype(float), degrees=True)
center = df.loc['sMPA',['X','Y','Z']].values.astype(float)+rotmat.apply(coords[location])

center

array([ 102.02180377,  248.61238541, 1268.61637067])

# Compute the position of the projector that shoots that location through the sMask

In [12]:
uvec = df.loc['sMask',['X','Y','Z']].values.astype(float) - center
uvec /= np.linalg.norm(uvec)
df.loc[location,['X','Y','Z']] = center+uvec*focal_length
df.loc[location,['color']] = 'royalblue'
df.loc[location,['uvec_X','uvec_Y','uvec_Z']] = -uvec*focal_length

update_RxRy(df,location,GSA_angle_WCS_deg)


array([-0.1435233 , -0.19044666, -0.97114939])

In [40]:
def create_SORC_coords(df,fields):
    for field in fields.index:
        coords = fields.loc[field,['X','Y']].values.astype(float)
        coords = np.append(coords,[0.])
        rotmat = R.from_euler('XYZ',df.loc['sMPA',['Rx','Ry','Rz']].values.astype(float), degrees=True)
        center = df.loc['sMPA',['X','Y','Z']].values.astype(float)+rotmat.apply(coords)
        uvec = df.loc['sMask',['X','Y','Z']].values.astype(float) - center
        uvec /= np.linalg.norm(uvec)
        df.loc[field,['X','Y','Z']] = center+uvec*focal_length
        df.loc[field,['color']] = 'royalblue'
        df.loc[field,['uvec_X','uvec_Y','uvec_Z']] = -uvec*focal_length

        update_RxRy(df,field,GSA_angle_WCS_deg)


In [16]:
df

X           Y           Z         Rx         Ry      color  \
Name                                                                          
PR1       0.991107    0.115821    0.754433   0.021212   0.045726    crimson   
PR2    -109.485707  -46.959858    23.47922  -4.915799  10.693732    crimson   
PR3     111.329338  -46.602151   23.152164  -4.873486 -10.586882    crimson   
PR4     103.162412   58.543344   31.771468   5.291407 -10.304568    crimson   
PR5    -101.345145   58.305984   31.898431   5.265896  10.417492    crimson   
sMask        1.469  115.184948  588.226533        NaN        NaN          g   
sMPA         2.022     248.477    1268.761 -24.302000   0.043600     purple   
center  -83.427951    2.531871   13.771912   0.008344   8.251776  royalblue   

            uvec_X      uvec_Y       uvec_Z      Rz  
Name                                                 
PR1       1.031195  248.370010  1268.027256     NaN  
PR2     239.765216  350.383100  1220.379520     NaN  
PR3    -237.396982  349.604227  1221.065684     NaN  
PR4    -231.135803  128.737997  1264.746858     NaN  
PR5     233.640931  129.254531  1264.233772     NaN  
sMask          NaN         NaN          NaN     NaN  
sMPA      0.000761    0.411546     0.911389  0.1048  
center  185.449755  246.080514  1254.844459     NaN

In [17]:
ax = plot_poses(df)
plot_sMPA(df, ax)
df

<IPython.core.display.Javascript object>

X           Y           Z         Rx         Ry      color  \
Name                                                                          
PR1       0.991107    0.115821    0.754433   0.021212   0.045726    crimson   
PR2    -109.485707  -46.959858    23.47922  -4.915799  10.693732    crimson   
PR3     111.329338  -46.602151   23.152164  -4.873486 -10.586882    crimson   
PR4     103.162412   58.543344   31.771468   5.291407 -10.304568    crimson   
PR5    -101.345145   58.305984   31.898431   5.265896  10.417492    crimson   
sMask        1.469  115.184948  588.226533        NaN        NaN          g   
sMPA         2.022     248.477    1268.761 -24.302000   0.043600     purple   
center  -83.427951    2.531871   13.771912   0.008344   8.251776  royalblue   

            uvec_X      uvec_Y       uvec_Z      Rz  
Name                                                 
PR1       1.031195  248.370010  1268.027256     NaN  
PR2     239.765216  350.383100  1220.379520     NaN  
PR3    -237.396982  349.604227  1221.065684     NaN  
PR4    -231.135803  128.737997  1264.746858     NaN  
PR5     233.640931  129.254531  1264.233772     NaN  
sMask          NaN         NaN          NaN     NaN  
sMPA      0.000761    0.411546     0.911389  0.1048  
center  185.449755  246.080514  1254.844459     NaN

In [18]:
check_pupil_crossing([df])

        sMask to chief ray (mm)  sMask to origin (mm)
PR1                2.986135e-04            598.635616
PR2                2.329674e-04            597.947667
PR3                3.162577e-04            597.957692
PR4                3.660392e-04            568.499833
PR5                3.068644e-04            568.600846
sMask                       NaN              0.000000
sMPA               1.585905e+02            693.465383
center             2.580453e-14            591.520346


In [23]:
dat = pd.read_excel('/Users/mrizzo/Box Sync/WFI/I&T/TPT Input Information_v9.xlsx',sheet_name='Fields',skiprows=3,index_col='Name')
dat

Unnamed: 0                           Description  \
Name                                                                        
CenterField        FDPR, Dispersion  As close as possible to field center   
SCA1                            NaN                            SCA center   
SCA2                            NaN                            SCA center   
SCA3                            NaN                            SCA center   
SCA4                            NaN                            SCA center   
SCA5                            NaN                            SCA center   
SCA6                            NaN                            SCA center   
SCA7                            NaN                            SCA center   
SCA8                            NaN                            SCA center   
SCA9                            NaN                            SCA center   
SCA10                           NaN                            SCA center   
SCA11                           NaN                            SCA center   
SCA12                           NaN                            SCA center   
SCA13                           NaN                            SCA center   
SCA14                           NaN                            SCA center   
SCA15                           NaN                            SCA center   
SCA16                           NaN                            SCA center   
SCA17                           NaN                            SCA center   
SCA18                           NaN                            SCA center   
SL1A         Stray light + ghosting                              SCA1 gap   
SL1B                            NaN                     SCA2 light shield   
SL2                             NaN       SCA2 gap -purge port reflection   
SL3                             NaN                              SCA3 gap   
SL4                             NaN                      SCA4 shield edge   
SL6A                            NaN            SCA6 shield light trap gap   
SL6B                            NaN                     SCA6 diving board   
SL6C                            NaN                      SCA6 underside 1   
SL6D                            NaN                      SCA6 underside 2   
SL6E                            NaN                      SCA6 underside 3   
SL9A                            NaN                   fiber optic ferrule   
SL9B                            NaN                 SCA9 light trap gap 1   
SL9C                            NaN    SCA9 light trap gap 2; ghost field   
SL9D                            NaN                            SCA9 fiber   
SL9E                            NaN                      SCA9 fiber glint   
GHOST9W                         NaN  Ghost SCA9 in field for W146 & grism   
GHOST9G                         NaN         Ghost SCA9 in field for grism   
FOV7A                           FOV                             FOV edges   
FOV7B                           NaN                             FOV edges   
FOV9                            NaN                             FOV edges   
FOV6                            NaN                             FOV edges   
FOV3A                           NaN                             FOV edges   
FOV3B                           NaN                             FOV edges   
FOV12A                          NaN                             FOV edges   
FOV12B                          NaN                             FOV edges   
FOV15                           NaN                             FOV edges   
FOV18                           NaN                             FOV edges   
FOV16A                          NaN                             FOV edges   
FOV16B                          NaN                             FOV edges   
FOV13                           NaN                             FOV edges   
FOV10                           NaN                             FOV edges   
FOV1                 

In [27]:
for location in dat.index:
    dat.loc[field,['X','Y']].values.astype(float)

[-4.  0.]
[-22.14  12.15]
[-22.29 -37.03]
[-22.44 -82.06]
[-66.42  20.9 ]
[-66.92 -28.28]
[-67.42 -73.06]
[-110.7   42.2]
[-111.48   -6.98]
[-112.64  -51.06]
[22.14 12.15]
[ 22.29 -37.03]
[ 22.44 -82.06]
[66.42 20.9 ]
[ 66.92 -28.28]
[ 67.42 -73.06]
[110.7  42.2]
[111.48  -6.98]
[112.64 -51.06]
[0. 0.]
[-21.78  -10.675]
[  0.  -33.9]
[  0.   -82.06]
[-75.  49.]
[-75.  -98.2]
[ -75. -104.]
[-67.33 -93.78]
[-67.33 -93.98]
[-67.33 -94.18]
[-140.  -52.]
[-92. -52.]
[-90. -50.]
[-106.31  -80.78]
[-107.02  -81.48]
[-95. -50.]
[-98. -50.]
[-130.14   61.64]
[-91.26  61.64]
[-132.08  -70.5 ]
[-86.86 -92.5 ]
[  -2.7 -101.5]
[ -41.88 -101.5 ]
[  41.88 -101.5 ]
[   2.7 -101.5]
[ 86.86 -92.5 ]
[132.08 -70.5 ]
[130.14  61.64]
[91.26 61.64]
[46.98 40.34]
[ 2.7  31.59]
[-2.7  31.59]
[-46.98  40.34]


In [41]:
create_SORC_coords(df,dat)
df

X           Y           Z         Rx         Ry  \
Name                                                                    
PR1            0.991107    0.115821    0.754433   0.021212   0.045726   
PR2         -109.485707  -46.959858    23.47922  -4.915799  10.693732   
PR3          111.329338  -46.602151   23.152164  -4.873486 -10.586882   
PR4          103.162412   58.543344   31.771468   5.291407 -10.304568   
PR5         -101.345145   58.305984   31.898431   5.265896  10.417492   
sMask             1.469  115.184948  588.226533        NaN        NaN   
sMPA              2.022     248.477    1268.761 -24.302000   0.043600   
center       -83.427951    2.531871   13.771912   0.008344   8.251776   
CenterField    4.444613    0.125367    0.750715   0.022152  -0.284795   
SCA1          20.258702  -10.318189    0.805459  -0.956486  -1.791674   
SCA2          19.615615   31.848909     5.53254   2.964336  -1.765808   
SCA3          18.901546   68.290577   15.340906   6.423905  -1.737130   
SCA4          58.587528  -16.766965    5.793263  -1.661539  -5.463433   
SCA5          56.922901   25.352245    9.338281   2.282616  -5.407551   
SCA6          55.009563   61.739427   18.027187   5.748738  -5.340955   
SCA7           96.47304  -32.722578   16.492607  -3.400922  -9.138997   
SCA8          94.121739    9.278758   17.203352   0.596390  -9.064394   
SCA9          91.625896   45.466941   23.335402   4.067721  -9.000795   
SCA10        -18.244733  -10.376592    0.831785  -0.962459   1.879764   
SCA11        -17.718214   31.794229    5.557631   2.958721   1.867051   
SCA12        -17.092199   68.239539   15.364785   6.418641   1.849586   
SCA13        -56.550499   -16.94262    5.872565  -1.679658   5.549434   
SCA14        -55.004673   25.187327    9.413996   2.265540   5.506850   
SCA15        -53.181424   61.585041   18.099391   5.732684   5.451553   
SCA16        -94.379132  -33.017841   16.626249  -3.431906   9.219656   
SCA17        -92.152178    9.000283   17.331227   0.567073   9.158701   
SCA18        -89.751165   45.204115   23.458087   4.039946   9.106586   
SL1A           0.991604    0.116042     0.73197   0.021621   0.045690   
SL1B          19.614317    9.488297    2.167739   0.880014  -1.745264   
SL2            0.951997   29.106135    4.409642   2.716156   0.050196   
SL3            0.904496   68.213396   14.722068   6.421273   0.056208   
SL4           66.907773  -40.955903    8.365193  -3.967230  -6.219037   
SL6A          59.347387   81.134468   26.288393   7.635919  -5.869911   
SL6B          58.963349   85.448376   28.089105   8.064635  -5.852320   
SL6C          53.754048   77.716264   23.635271   7.306675  -5.279347   
SL6D          53.742306   77.867529   23.694367   7.321564  -5.278808   
SL6E          53.730558   78.018736   23.753553   7.336450  -5.278268   
SL9A          111.87208   47.279846   31.949406   4.143801 -11.144858   
SL9B          75.630094   45.550869   18.274202   4.137894  -7.359448   
SL9C          74.201293   43.880808   17.439032   3.982862  -7.206334   
SL9D          84.300786   68.497069   28.271362   6.337323  -8.385817   
SL9E          84.768547   69.046999   28.636476   6.390150  -8.438553   
GHOST9W       78.114898   44.031345   18.594181   3.983479  -7.604909   
GHOST9G       80.451166   44.125207   19.315709   3.983850  -7.843698   
FOV7A        113.122985  -47.771813   24.053087  -5.007417 -10.765413   
FOV7B         81.185228  -51.012553   13.042617  -5.012976  -7.584087   
FOV9         104.228459   61.332779   33.175848   5.561889 -10.440772   
FOV6          68.739354   77.018424   26.926226   7.213568  -6.818555   
FOV3A          3.010662   83.128885   20.473033   7.871941  -0.155332   
FOV3B         33.683373   83.293012   22.599936   7.876388  -3.254512   
FOV12A       -31.905173   83.201306   22.643278   7.866880   3.371643   
FOV12B        -1.233878    83.12295   20.475834   7.871328   0.272331   
FOV15        -66.942277   76.827508   27.016519   7.193543   6.934024   
FOV18  

In [42]:
ax = plot_poses(df)
plot_sMPA(df, ax)
df

<IPython.core.display.Javascript object>

X           Y           Z         Rx         Ry  \
Name                                                                    
PR1            0.991107    0.115821    0.754433   0.021212   0.045726   
PR2         -109.485707  -46.959858    23.47922  -4.915799  10.693732   
PR3          111.329338  -46.602151   23.152164  -4.873486 -10.586882   
PR4          103.162412   58.543344   31.771468   5.291407 -10.304568   
PR5         -101.345145   58.305984   31.898431   5.265896  10.417492   
sMask             1.469  115.184948  588.226533        NaN        NaN   
sMPA              2.022     248.477    1268.761 -24.302000   0.043600   
center       -83.427951    2.531871   13.771912   0.008344   8.251776   
CenterField    4.444613    0.125367    0.750715   0.022152  -0.284795   
SCA1          20.258702  -10.318189    0.805459  -0.956486  -1.791674   
SCA2          19.615615   31.848909     5.53254   2.964336  -1.765808   
SCA3          18.901546   68.290577   15.340906   6.423905  -1.737130   
SCA4          58.587528  -16.766965    5.793263  -1.661539  -5.463433   
SCA5          56.922901   25.352245    9.338281   2.282616  -5.407551   
SCA6          55.009563   61.739427   18.027187   5.748738  -5.340955   
SCA7           96.47304  -32.722578   16.492607  -3.400922  -9.138997   
SCA8          94.121739    9.278758   17.203352   0.596390  -9.064394   
SCA9          91.625896   45.466941   23.335402   4.067721  -9.000795   
SCA10        -18.244733  -10.376592    0.831785  -0.962459   1.879764   
SCA11        -17.718214   31.794229    5.557631   2.958721   1.867051   
SCA12        -17.092199   68.239539   15.364785   6.418641   1.849586   
SCA13        -56.550499   -16.94262    5.872565  -1.679658   5.549434   
SCA14        -55.004673   25.187327    9.413996   2.265540   5.506850   
SCA15        -53.181424   61.585041   18.099391   5.732684   5.451553   
SCA16        -94.379132  -33.017841   16.626249  -3.431906   9.219656   
SCA17        -92.152178    9.000283   17.331227   0.567073   9.158701   
SCA18        -89.751165   45.204115   23.458087   4.039946   9.106586   
SL1A           0.991604    0.116042     0.73197   0.021621   0.045690   
SL1B          19.614317    9.488297    2.167739   0.880014  -1.745264   
SL2            0.951997   29.106135    4.409642   2.716156   0.050196   
SL3            0.904496   68.213396   14.722068   6.421273   0.056208   
SL4           66.907773  -40.955903    8.365193  -3.967230  -6.219037   
SL6A          59.347387   81.134468   26.288393   7.635919  -5.869911   
SL6B          58.963349   85.448376   28.089105   8.064635  -5.852320   
SL6C          53.754048   77.716264   23.635271   7.306675  -5.279347   
SL6D          53.742306   77.867529   23.694367   7.321564  -5.278808   
SL6E          53.730558   78.018736   23.753553   7.336450  -5.278268   
SL9A          111.87208   47.279846   31.949406   4.143801 -11.144858   
SL9B          75.630094   45.550869   18.274202   4.137894  -7.359448   
SL9C          74.201293   43.880808   17.439032   3.982862  -7.206334   
SL9D          84.300786   68.497069   28.271362   6.337323  -8.385817   
SL9E          84.768547   69.046999   28.636476   6.390150  -8.438553   
GHOST9W       78.114898   44.031345   18.594181   3.983479  -7.604909   
GHOST9G       80.451166   44.125207   19.315709   3.983850  -7.843698   
FOV7A        113.122985  -47.771813   24.053087  -5.007417 -10.765413   
FOV7B         81.185228  -51.012553   13.042617  -5.012976  -7.584087   
FOV9         104.228459   61.332779   33.175848   5.561889 -10.440772   
FOV6          68.739354   77.018424   26.926226   7.213568  -6.818555   
FOV3A          3.010662   83.128885   20.473033   7.871941  -0.155332   
FOV3B         33.683373   83.293012   22.599936   7.876388  -3.254512   
FOV12A       -31.905173   83.201306   22.643278   7.866880   3.371643   
FOV12B        -1.233878    83.12295   20.475834   7.871328   0.272331   
FOV15        -66.942277   76.827508   27.016519   7.193543   6.934024   
FOV18  

In [48]:
df_encoder_space = pd.DataFrame(columns=['X', 'Y', 'Z', 'Rx', 'Ry'])
for pose in df.index:
    df_encoder_space = pd.concat([df_encoder_space,calculate_encoders_from_5DOF(pd.DataFrame(df.loc[pose]).T)])

df_encoder_space

X             Y            Z         Rx         Ry
PR1             0.968110      0.149959     0.739956   0.021941   0.047806
PR2          -109.388136    -47.030270    23.376694  -4.915767  10.692735
PR3           111.283617    -46.304538    23.248360  -4.872000 -10.586625
PR4           102.971310     58.768453    31.962197   5.296265 -10.302126
PR5          -101.400777     58.164365    31.901152   5.265307  10.418327
sMask                NaN           NaN          NaN        NaN        NaN
sMPA         4584.702534 -14968.000998  8292.768964 -24.535917  -0.026506
center        -83.407012      2.407195    13.737918   0.008965   8.252468
CenterField     4.420049      0.165764     0.739493   0.022899  -0.282702
SCA1           20.242596    -10.252600     0.805087  -0.955457  -1.789634
SCA2           19.542735     31.901198     5.569214   2.964698  -1.763381
SCA3           18.777673     68.335800    15.411971   6.426134  -1.733327
SCA4           58.559345    -16.630549     5.842258  -1.659918  -5.461654
SCA5           56.832742     25.487434     9.416176   2.283860  -5.405397
SCA6           54.872056     61.848325    18.143965   5.751295  -5.337690
SCA7           96.425506    -32.470586    16.578586  -3.398667  -9.137992
SCA8           94.003938      9.550454    17.306750   0.599340  -9.062853
SCA9           91.464126     45.689650    23.483901   4.070905  -8.998648
SCA10         -18.243817    -10.383738     0.799356  -0.961554   1.881660
SCA11         -17.773757     31.774114     5.562902   2.958678   1.869304
SCA12         -17.203888     68.231357    15.397478   6.420216   1.853191
SCA13         -56.519135    -17.026680     5.827627  -1.678555   5.550723
SCA14         -55.032454     25.094254     9.403239   2.265426   5.508431
SCA15         -53.270665     61.515562    18.109206   5.733147   5.454172
SCA16         -94.305876    -33.120337    16.554256  -3.430890   9.219667
SCA17         -92.134826      8.865401    17.301185   0.567397   9.159118
SCA18         -89.793203     45.069637    23.455028   4.039279   9.107487
SL1A            0.968597      0.149783     0.717811   0.022350   0.047770
SL1B           19.570100      9.555232     2.179685   0.880684  -1.743094
SL2             0.891451     29.125023     4.424471   2.716289   0.052538
SL3             0.787589     68.231318    14.771903   6.423134   0.059956
SL4            66.919588    -40.851298     8.433142  -3.966453  -6.217798
SL6A           59.180031     81.248606    26.430240   7.641533  -5.865316
SL6B           58.791120     85.560707    28.234378   8.071182  -5.847308
SL6C           53.595229     77.822532    23.764148   7.311392  -5.274998
SL6D           53.583288     77.973798    23.823345   7.326308  -5.274446
SL6E           53.571342     78.125005    23.882632   7.341221  -5.273894
SL9A          111.676622     47.558666    32.132264   4.148514 -11.143018
SL9B           75.491513     45.722475    18.401672   4.140215  -7.357068
SL9C           74.066773     44.049643    17.562475   3.985049  -7.203985
SL9D           84.124303     68.663269    28.444077   6.342067  -8.382506
SL9E           84.591045     69.213020    28.810987   6.394997  -8.435213
GHOST9W        77.974939     44.212027    18.722972   3.985860  -7.602606
GHOST9G        80.307824     44.313150    19.447676   3.986351  -7.841424
FOV7A         113.076388    -47.466645    24.149714  -5.006020 -10.765248
FOV7B          81.203280    -50.892383    13.133487  -5.012994  -7.583414
FOV9          104.034453     61.549963    33.375457   5.567109 -10.438213
FOV6           68.569423     77.147880    27.081362   7.218824  -6.814415
FOV3A           2.866470     83.173758    20.527869   7.876230  -0.150404
FOV3B          33.528265     83.374795    22.698705   7.881499  -3.249546
FOV12A        -32.043000     83.207360    22.662023   7.870535   3.376208
FOV12B         -1.377071     83.162964    20.525472   7.875528   0.277233
FOV15         -67.057737     76.773866    27.018396   7.195466   6.937234
FOV18        -102.445958   

In [49]:
df_encoder_space.to_excel('/Users/mrizzo/Box Sync/WFI/I&T/fields_encoders.xlsx')